In [1]:
!pip install scrapy
!pip install Twisted==22.10.0

In [2]:
import os 
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
from bs4 import BeautifulSoup
from urllib.parse import urlparse, unquote

In [3]:
class FullPageSpider(scrapy.Spider):
    name='FullPageSpider'
    allowed_domains= ['lloydsbanking.com', 'lloydsbank.com']
    start_urls=['https://www.lloydsbank.com/business/help-and-support/online-banking.html']
        
    custom_settings = {'USER_AGENT' : 'Mozilla/5.0 (Windoes NT 10.0; Win64; x64) AppleWebKit537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',}
    
    def parse(self, response):
        self.save_page(response)
        
        link_extractor= LinkExtractor(allow=r'/business/help-and-support/')
        for link in link_extractor.extract_links(response):
            yield scrapy.Request(link.url, callback=self.parse)
            
    def save_page (self, response):
        output_dir ='output'
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
            
        parsed_url = urlparse(unquote(response.url))
        filename=os.path.basename(parsed_url.path)
        if filename.endswith('.html'):
            file_split= os.path.splitext(filename)
            filename= f'{file_split[0]}.txt'
        else:
            filename= f'{file_split}.txt'
            
        file_path= os.path.join(output_dir, filename)
        
        soup= BeautifulSoup(response.body, 'html.parser')
        
        nested_selector= 'body .main .content > div:nth-of-type(2) > dice .container-fluid .row > div:nth-of-type(2)'
        
        with open(file_path,'a', encoding='utf-8') as file:
            for div in soup.select(nested_selector):
                content = div.get_text(strip=False)
                file.write(content + '\n')
                
        if (os.path.getsize('output/'+ filename) == 0):
            nested_selector='body .main .content > div > div .container-fluid .row > div:nth-of-type(2)'
            with open(file_path,'a', encoding='utf-8') as file:
                for div in soup.select(nested_selector):
                    content = div.get_text(strip=True)
                    file.write(content + '\n')
            
        if (os.path.getsize('output/'+ filename) == 0):
            nested_selector='body .main .content > div > div .container-fluid .row > div'
            with open(file_path,'a', encoding='utf-8') as file:
                for div in soup.select(nested_selector):
                    content = div.get_text(strip=True)
                    file.write(content + '\n')
            
        self.log(f'Saved file {file_path}')

In [4]:
process= CrawlerProcess()
process.crawl(FullPageSpider)
process.start()

2023-11-30 17:14:57 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-11-30 17:14:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) - [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.3 19 Sep 2023), cryptography 38.0.2, Platform Linux-4.19.0-25-cloud-amd64-x86_64-with-debian-10.13
2023-11-30 17:14:57 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (Windoes NT 10.0; Win64; x64) AppleWebKit537.36 '
               '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
2023-11-30 17:14:57 [py.warnings] WARNING: /opt/conda/lib/python3.7/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for t